conda install -c conda-forge ipython-sql

## Introduction and Schema Diagram

In [2]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

## Overview of Data

In [3]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
Album,table
Artist,table
Customer,table
Employee,table
Genre,table
Invoice,table
InvoiceLine,table
MediaType,table
Playlist,table
PlaylistTrack,table


## Selecting New Albums to Purchase¶
Specifically we will write a query to find out which genres sell the most tracks in the USA, write up a summary, and make a recommendation for the three artists whose albums we should purchase for the store.

In [4]:
%%sql

WITH usa_tracks_sold AS
   (
    SELECT il.* FROM InvoiceLine il
    INNER JOIN Invoice i on il.InvoiceId = i.InvoiceId
    INNER JOIN Customer c on i.CustomerId = c.CustomerId
    WHERE c.Country = "USA"
   )

SELECT
    g.name genre,
    count(uts.InvoiceLineId) tracks_sold,
    cast(count(uts.InvoiceLineId) AS FLOAT) / (
        SELECT COUNT(*) from usa_tracks_sold
    ) percentage_sold
FROM usa_tracks_sold uts
INNER JOIN Track t on t.TrackId = uts.TrackId
INNER JOIN Genre g on g.GenreId = t.GenreId
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10;

 * sqlite:///chinook.db
Done.


genre,tracks_sold,percentage_sold
Rock,157,0.31781376518218624
Latin,91,0.18421052631578946
Metal,64,0.12955465587044535
Alternative & Punk,50,0.10121457489878542
Jazz,22,0.044534412955465584
Blues,15,0.030364372469635626
TV Shows,14,0.02834008097165992
R&B/Soul,12,0.024291497975708502
Comedy,8,0.016194331983805668
Classical,8,0.016194331983805668


Based on the sales of tracks across different genres in the USA, we should purchase the new albums by the following artists:

- Red Tone (Punk)
- Slim Jim Bites (Blues)
- Meteor and the Girls (Pop)

It's worth keeping in mind that combined, these three genres only make up only 17% of total sales, so we should be on the lookout for artists and albums from the 'rock' genre, which accounts for 53% of sales.

## Analyzing Employee Sales Performance

In [5]:
%%sql

Select *
From Employee
LIMIT 5

 * sqlite:///chinook.db
Done.


EmployeeId,LastName,FirstName,Title,ReportsTo,BirthDate,HireDate,Address,City,State,Country,PostalCode,Phone,Fax,Email
1,Adams,Andrew,General Manager,None,1962-02-18 00:00:00,2002-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
2,Edwards,Nancy,Sales Manager,1,1958-12-08 00:00:00,2002-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
3,Peacock,Jane,Sales Support Agent,2,1973-08-29 00:00:00,2002-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
4,Park,Margaret,Sales Support Agent,2,1947-09-19 00:00:00,2003-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
5,Johnson,Steve,Sales Support Agent,2,1965-03-03 00:00:00,2003-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com


In [6]:
%%sql

WITH customer_support_rep_sales AS

    (
    SELECT 
        i.CustomerId,
        c.SupportRepId,
        ROUND(SUM(i.Total), 2) total
    FROM Invoice i
    INNER JOIN Customer c ON i.CustomerId = c.CustomerId
    GROUP BY 1,2
    )
    
SELECT    
    e.FirstName || " " || e.LastName employee,
    e.HireDate,
    SUM(csrs.total) total_sales
FROM customer_support_rep_sales csrs
INNER JOIN Employee e ON e.EmployeeID = csrs.SupportRepId
GROUP BY 1;

 * sqlite:///chinook.db
Done.


employee,HireDate,total_sales
Jane Peacock,2002-04-01 00:00:00,833.04
Margaret Park,2003-05-03 00:00:00,775.4
Steve Johnson,2003-10-17 00:00:00,720.16


While there is a 20% difference in sales between Jane (the top employee) and Steve (the bottom employee), the difference roughly corresponds with the differences in their hiring dates.

## Analyzing Sales by Country

Write a query that finds the total dollar amount of sales assigned to each sales support agent within the company. Add any extra attributes for that employee that you find are relevant to the analysis.

In [13]:
%%sql

WITH country_or_other AS
    (
     SELECT
       CASE
           WHEN (
                 SELECT count(*)
                 FROM Customer
                 where Country = c.Country
                ) = 1 THEN "Other"
           ELSE c.Country
       END AS country,
       c.CustomerId,
       il.*
     FROM InvoiceLine il
     INNER JOIN Invoice i ON i.InvoiceId = il.InvoiceId
     INNER JOIN Customer c ON c.CustomerId = i.CustomerId
    )
    
SELECT
    country,
    customers,
    total_sales,
    average_order,
    customer_lifetime_value
FROM
    (
    SELECT
        country,
        count(distinct CustomerId) customers,
        SUM(UnitPrice) total_sales,
        SUM(UnitPrice) / count(distinct CustomerId) customer_lifetime_value,
        SUM(UnitPrice) / count(distinct InvoiceId) average_order,
        CASE
            WHEN country = "Other" THEN 1
            ELSE 0
        END AS sort
    FROM country_or_other
    GROUP BY country
    ORDER BY sort ASC, total_sales DESC
    );    

 * sqlite:///chinook.db
Done.


country,customers,total_sales,average_order,customer_lifetime_value
USA,13,523.060000000003,5.747912087912121,40.235384615384845
Canada,8,303.96000000000134,5.4278571428571665,37.99500000000017
France,5,195.1000000000003,5.574285714285723,39.02000000000006
Brazil,5,190.10000000000028,5.4314285714285795,38.02000000000005
Germany,4,156.48,5.588571428571428,39.12
United Kingdom,3,112.85999999999979,5.374285714285704,37.619999999999926
Czech Republic,2,90.23999999999997,6.445714285714283,45.11999999999998
Portugal,2,77.23999999999997,5.517142857142855,38.61999999999998
India,2,75.25999999999999,5.789230769230769,37.629999999999995
Other,15,604.3000000000038,5.7552380952381315,40.286666666666925


Based on the data, there may be opportunity in the following countries:

- Czech Republic
- United Kingdom
- India

It's worth keeping in mind that because the amount of data from each of these countries is relatively low. Because of this, we should be cautious spending too much money on new marketing campaigns, as the sample size is not large enough to give us high confidence. A better approach would be to run small campaigns in these countries, collecting and analyzing the new customers to make sure that these trends hold with new customers.

## Albums vs Individual Tracks

Write a query that categorizes each invoice as either an album purchase or not, and calculates the following summary statistics:

- Number of invoices
- Percentage of invoices

In [15]:
%%sql

WITH invoice_first_track AS
    (
     SELECT
         il.InvoiceId InvoiceId,
         MIN(il.TrackId) first_track_id
     FROM InvoiceLine il
     GROUP BY 1
    )
SELECT
    album_purchase,
    COUNT(InvoiceId) number_of_invoices,
    CAST(count(InvoiceId) AS FLOAT) / (
                                         SELECT COUNT(*) FROM Invoice
                                      ) percent  
FROM
    (
    SELECT
        ifs.*,
        CASE
            WHEN
                 (
                  SELECT t.TrackId FROM Track t
                  WHERE t.AlbumId = (
                                      SELECT t2.AlbumId FROM Track t2
                                      WHERE t2.TrackId = ifs.first_track_id
                                     ) 

                  EXCEPT 

                  SELECT il2.TrackId FROM InvoiceLine il2
                  WHERE il2.InvoiceId = ifs.InvoiceId
                 ) IS NULL
             AND
                 (
                  SELECT il2.TrackId FROM InvoiceLine il2
                  WHERE il2.InvoiceId = ifs.InvoiceId

                  EXCEPT 

                  SELECT t.TrackId FROM Track t
                  WHERE t.AlbumId = (
                                      SELECT t2.AlbumId FROM Track t2
                                      WHERE t2.trackId = ifs.first_track_id
                                     ) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END AS "album_purchase"
     FROM invoice_first_track ifs
    )
GROUP BY album_purchase;    

 * sqlite:///chinook.db
Done.


album_purchase,number_of_invoices,percent
no,410,0.9951456310679612
yes,2,0.0048543689320388345
